## Importing

In [1]:
import fiona
from pathlib import Path
import pandas as pd
from osgeo import ogr, gdal, gdalconst, osr
from osgeo_utils import gdal_merge
import numpy as np


## Paths

In [2]:
tiff_path = Path(r'data/tiff')
shp_path = Path(r'data/shp')
for f in shp_path.glob('*.shp'):
    print(f.name)

XQ_bd_amz_25km.shp
Dist_bd_amz_25km.shp
XArDS_bd_amz_25km.shp
DeAr_bd_amz_25km.shp
XDeDS_bd_amz_25km.shp
ArDS_bd_amz_25km.shp
DS_bd_amz_25km.shp
AcAr_bd_amz_25km.shp
Dvd_bd_amz_25km.shp
CtDS_bd_amz_25km.shp
DryMonths_bd_amz_25km.shp
Nuvem_bd_amz_25km.shp
EF_bd_amz_25km.shp
OcDS_bd_amz_25km.shp


## Define Biweekly data

In [3]:
mask = Path(r'data/shp/ArDS_bd_amz_25km.shp')
biweekly = {
    'ArDS': Path(r'data/shp/ArDS_bd_amz_25km.shp'),
    'OcDS': Path(r'data/shp/OcDS_bd_amz_25km.shp'),
    'CtDS': Path(r'data/shp/CtDS_bd_amz_25km.shp'),
    'DeAr': Path(r'data/shp/DeAr_bd_amz_25km.shp'),
    'AcAr': Path(r'data/shp/AcAr_bd_amz_25km.shp'),
    'XQ': Path(r'data/shp/XQ_bd_amz_25km.shp'),
    'nv': Path(r'data/shp/Nuvem_bd_amz_25km.shp'),
}
quartely = {
    'DS': Path(r'data/shp/DS_bd_amz_25km.shp'),
    'XArDS': Path(r'data/shp/XArDS_bd_amz_25km.shp'),
    'XDeDS': Path(r'data/shp/XDeDS_bd_amz_25km.shp'),
}
fixed = {
    'Dist': Path(r'data/shp/Dist_bd_amz_25km.shp'),
    'EF': Path(r'data/shp/EF_bd_amz_25km.shp'),
    'DryMonths': Path(r'data/shp/DryMonths_bd_amz_25km.shp'),
    'Dvd': Path(r'data/shp/Dvd_bd_amz_25km.shp'),
}

## Mask data

In [47]:
cells_path = mask
v_cells_data = ogr.Open(str(cells_path))
l_cells_data = v_cells_data.GetLayer()
x_res = 25000
y_res = 25000
x_min, x_max, y_min, y_max = l_cells_data.GetExtent()

target_ds = gdal.Rasterize(
    str(tiff_path / f'mask.tif'),
    str(cells_path),
    xRes = x_res,
    yRes = y_res,
    burnValues  = [1],
    outputBounds=[x_min, y_min, x_max, y_max],
    initValues  = [0],
    outputType=gdal.GDT_Byte
)
target_ds = None

## Coordinates Map

In [5]:
cells_path = mask
col = fiona.open(cells_path)
attr_dict = {att:att for att in col.schema['properties'].keys() if str(att) in ['col', 'row']}
sorted_attr_dict = dict(sorted(attr_dict.items()))
    
no_data_value = -1
v_cells_data = ogr.Open(str(cells_path))
l_cells_data = v_cells_data.GetLayer()
x_res = 25000
y_res = 25000
x_min, x_max, y_min, y_max = l_cells_data.GetExtent()

count = 0
list_files = []
for i, attr in enumerate(sorted_attr_dict):
    #if i < 10:
    #    continue
    count +=1
    target_ds = gdal.Rasterize(
        str(tiff_path / f'{sorted_attr_dict[attr]}.tif'),
        str(cells_path),
        xRes = x_res,
        yRes = y_res,
        attribute  = sorted_attr_dict[attr],
        outputBounds=[x_min, y_min, x_max, y_max],
        noData = no_data_value,
        outputType=gdal.GDT_Float32
    )
    list_files.append(str(tiff_path / f'{sorted_attr_dict[attr]}.tif'))
    target_ds = None
    #print(f'{count:02d} - {sorted_attr_dict[attr]}')

parameters = ['', '-separate', '-a_nodata', f'{no_data_value}', '-o', f'data/tiff/Coordinates.tif'] + list_files
gdal_merge.main(parameters)
for f in list_files:
    fp = Path(f)
    fp.unlink()


0...10...20...30...40...50...60...70...80...90...100 - done.


## Biweekly Number

In [9]:
cells_path = mask
col = fiona.open(cells_path)
attr_dict = {f'{att[-2:]}{att[-4:-2]}{att[-6:-4]}':att for att in col.schema['properties'].keys() if str(att).startswith('ArDS')}
sorted_attr_dict = dict(sorted(attr_dict.items()))
    
no_data_value = 0
v_cells_data = ogr.Open(str(cells_path))
l_cells_data = v_cells_data.GetLayer()
x_res = 25000
y_res = 25000
x_min, x_max, y_min, y_max = l_cells_data.GetExtent()

count = 0
list_files = []
for i, attr in enumerate(sorted_attr_dict):
    #if i < 10:
    #    continue
    day = int(sorted_attr_dict[attr][-6:-4])
    month = int(sorted_attr_dict[attr][-4:-2])
    value = (month) / 12
    if day==1:
        value -= (1 / 24)
    count +=1
    target_ds = gdal.Rasterize(
        str(tiff_path / f'{sorted_attr_dict[attr]}.tif'),
        str(cells_path),
        xRes = x_res,
        yRes = y_res,
        burnValues  = [value],
        outputBounds=[x_min, y_min, x_max, y_max],
        noData = no_data_value,
        outputType=gdal.GDT_Float32
    )
    list_files.append(str(tiff_path / f'{sorted_attr_dict[attr]}.tif'))
    target_ds = None
    #print(f'{count:02d} - {sorted_attr_dict[attr]}')

parameters = ['', '-separate', '-a_nodata', f'{no_data_value}', '-o', f'data/tiff/Biweekly.tif'] + list_files
gdal_merge.main(parameters)
for f in list_files:
    fp = Path(f)
    fp.unlink()


0...10...20...30...40...50...60...70...80...90...100 - done.


## Biweekly Data

In [49]:
for data_att in biweekly:
    cells_path = biweekly[data_att]
    col = fiona.open(cells_path)
    att_len = len(data_att)
    attr_dict = {f'{att[-2:]}{att[-4:-2]}{att[-6:-4]}':att for att in col.schema['properties'].keys() if str(att).startswith(data_att)}
    sorted_attr_dict = dict(sorted(attr_dict.items()))
    if len(sorted_attr_dict) == 0:
        print(data_att)
        
    no_data_value = 0
    v_cells_data = ogr.Open(str(cells_path))
    l_cells_data = v_cells_data.GetLayer()
    x_res = 25000
    y_res = 25000
    x_min, x_max, y_min, y_max = l_cells_data.GetExtent()

    count = 0
    list_files = []
    for i, attr in enumerate(sorted_attr_dict):
        #if i < 10:
        #    continue
        count +=1
        target_ds = gdal.Rasterize(
            str(tiff_path / f'{sorted_attr_dict[attr]}.tif'),
            str(cells_path),
            xRes = x_res,
            yRes = y_res,
            attribute  = sorted_attr_dict[attr],
            outputBounds=[x_min, y_min, x_max, y_max],
            noData = no_data_value,
            outputType=gdal.GDT_Float32
        )
        list_files.append(str(tiff_path / f'{sorted_attr_dict[attr]}.tif'))
        target_ds = None
        #print(f'{count:02d} - {sorted_attr_dict[attr]}')

    parameters = ['', '-separate', '-a_nodata', f'{no_data_value}', '-o', f'data/tiff/{data_att}.tif'] + list_files
    gdal_merge.main(parameters)
    for f in list_files:
        fp = Path(f)
        fp.unlink()



0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


## Quaterly data

In [50]:
for data_att in quartely:
    cells_path = quartely[data_att]
    col = fiona.open(cells_path)
    att_len = len(data_att)
    attr_dict = {f'{att[-2:]}{att[-4:-2]}':att for att in col.schema['properties'].keys() if str(att).startswith(data_att)}
    sorted_attr_dict = dict(sorted(attr_dict.items()))
    if len(sorted_attr_dict) == 0:
        print(data_att)
        
    no_data_value = 0
    v_cells_data = ogr.Open(str(cells_path))
    l_cells_data = v_cells_data.GetLayer()
    x_res = 25000
    y_res = 25000
    x_min, x_max, y_min, y_max = l_cells_data.GetExtent()

    count = 0
    list_files = []
    for i, attr in enumerate(sorted_attr_dict):
        #if i < 10:
        #    continue
        count +=1
        target_ds = gdal.Rasterize(
            str(tiff_path / f'{sorted_attr_dict[attr]}.tif'),
            str(cells_path),
            xRes = x_res,
            yRes = y_res,
            attribute  = sorted_attr_dict[attr],
            outputBounds=[x_min, y_min, x_max, y_max],
            noData = no_data_value,
            outputType=gdal.GDT_Float32
        )
        list_files.append(str(tiff_path / f'{sorted_attr_dict[attr]}.tif'))
        target_ds = None
        #print(f'{count:02d} - {sorted_attr_dict[attr]}')

    parameters = ['', '-separate', '-a_nodata', f'{no_data_value}', '-o', f'data/tiff/{data_att}.tif'] + list_files
    gdal_merge.main(parameters)
    for f in list_files:
        fp = Path(f)
        fp.unlink()


0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


## Fixed Data

In [51]:
for data_att in fixed:
    cells_path = fixed[data_att]
    col = fiona.open(cells_path)
    att_len = len(data_att)
    attr_dict = {att:att for att in col.schema['properties'].keys() if not str(att) in ['id', 'col', 'row']}
    sorted_attr_dict = dict(sorted(attr_dict.items()))
    if len(sorted_attr_dict) == 0:
        print(data_att)
        
    no_data_value = 0
    v_cells_data = ogr.Open(str(cells_path))
    l_cells_data = v_cells_data.GetLayer()
    x_res = 25000
    y_res = 25000
    x_min, x_max, y_min, y_max = l_cells_data.GetExtent()

    count = 0
    list_files = []
    for i, attr in enumerate(sorted_attr_dict):
        #if i < 10:
        #    continue
        count +=1
        target_ds = gdal.Rasterize(
            str(tiff_path / f'{sorted_attr_dict[attr]}.tif'),
            str(cells_path),
            xRes = x_res,
            yRes = y_res,
            attribute  = sorted_attr_dict[attr],
            outputBounds=[x_min, y_min, x_max, y_max],
            noData = no_data_value,
            outputType=gdal.GDT_Float32
        )
        list_files.append(str(tiff_path / f'{sorted_attr_dict[attr]}.tif'))
        target_ds = None
        #print(f'{count:02d} - {sorted_attr_dict[attr]}')

    parameters = ['', '-separate', '-a_nodata', f'{no_data_value}', '-o', f'data/tiff/{data_att}.tif'] + list_files
    gdal_merge.main(parameters)
    for f in list_files:
        fp = Path(f)
        fp.unlink()


0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


## Count cells file

In [5]:
from features import mask_path
from utils.ops import load_sb_image, save_geotiff
import numpy as np
mask = load_sb_image(mask_path)
count_image = -1 * np.ones_like(mask)
shape = count_image.shape
count_image = count_image.flatten()
count_data = np.arange(mask.sum())
count_image[mask.flatten() == 1] = count_data
count_image = count_image.reshape(shape)
save_geotiff(mask_path, 'data/tiff/valid_cells.tif', count_image, 'float', -1)

In [4]:
count_data


array([0.000e+00, 1.000e+00, 2.000e+00, ..., 7.226e+03, 7.227e+03,
       7.228e+03])